In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
features_df = pd.read_csv('data/training_set_features.csv', index_col='id')
labels_df = pd.read_csv('data/training_set_labels.csv', index_col='id')
np.testing.assert_array_equal(features_df.index.values, labels_df.index.values)

In [3]:
numeric_cols = features_df.columns[features_df.dtypes != "object"].values
categorical_cols = features_df.columns[features_df.dtypes == "object"].values

In [4]:
# numeric_cols = [
#     'date_recorded',
#     'amount_tsh', 
#     'gps_height', 
#     'longitude', 
#     'latitude', 
#     'num_private',
#     'region_code', 
#     'district_code', 
#     'population', 
#     'construction_year'
# ]

# categorical_cols = [
#     'funder',
#     'installer',
#     'wpt_name',
#     'basin',
#     'subvillage',
#     'region',
#     'lga',
#     'ward',
#     'public_meeting',
#     'recorded_by',
#     'scheme_management',
#     'scheme_name', 
#     'permit', 
#     'extraction_type', 
#     'extraction_type_group',
#     'extraction_type_class', 
#     'management', 
#     'management_group',
#     'payment', 
#     'payment_type', 
#     'water_quality', 
#     'quality_group',
#     'quantity', 
#     'quantity_group', 
#     'source', 
#     'source_type',
#     'source_class', 
#     'waterpoint_type', 
#     'waterpoint_type_group'
# ]

In [5]:
numeric_cols

array(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'region_code', 'district_code', 'population', 'construction_year'],
      dtype=object)

In [6]:
categorical_cols

array(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting',
       'recorded_by', 'scheme_management', 'scheme_name', 'permit',
       'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group',
       'payment', 'payment_type', 'water_quality', 'quality_group',
       'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype=object)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from feature_engine.imputation import RandomSampleImputer
from sklearn.impute import MissingIndicator
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
#from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

from imblearn.over_sampling import ADASYN
from imblearn.pipeline import Pipeline 

RANDOM_SEED = 6    # Set a random seed for reproducibility!

In [8]:
missing_category_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value="Missing")
random_sample_imputer = RandomSampleImputer(random_state = 42)
scheme_management_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value="Company")
missing_indicator_imputer = MissingIndicator(error_on_new=True, features='missing-only')

imputer_categorical = ColumnTransformer(transformers=[
    ('missing_category', missing_category_imputer, ["funder", "installer", "scheme_name"]),
    ('random_sample', random_sample_imputer, ["public_meeting", "subvillage"]),
    ('scheme_management_imputer', scheme_management_imputer, ["scheme_management"]),
    ('missing_indicator_imputer', missing_indicator_imputer, ["permit"]),
], remainder="passthrough")

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('standard_scaler', StandardScaler()),
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    #('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_imputer', imputer_categorical),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

over_sampler = ADASYN(sampling_strategy="minority", random_state=42)

# 0.7812626262626263 score
# model = CatBoostClassifier(n_estimators=1000, learning_rate=0.05, rsm=1, random_strength=1, silent=False, loss_function='MultiClass',  random_state=42)

# n_estimators = 100 - score 0.8071717171717172
# n_estimators = 500 - score 0.8091077441077441
# n_estimators = 1000 - score 0.809090909090909
model = RandomForestClassifier(random_state=42, n_estimators = 500)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', over_sampler),
    ('model', model)
])

In [9]:
%%time

scores = cross_val_score(pipeline, features_df, labels_df["status_group"], cv=5, scoring='accuracy')
print(scores)

[0.81456229 0.80589226 0.81203704 0.80765993 0.80538721]
Wall time: 7min 37s


In [10]:
scores.mean()

0.8091077441077441

In [19]:
test_features_df = pd.read_csv("data/test_set_features.csv", index_col="id")

In [20]:
%%time

# Train model
pipeline.fit(features_df, labels_df["status_group"])

test_preds = pipeline.predict(test_features_df)

Wall time: 1min 59s


In [21]:
test_preds

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [22]:
submission_df = pd.read_csv("data/submission_format.csv", index_col="id")

In [23]:
np.testing.assert_array_equal(test_features_df.index.values, submission_df.index.values)

In [24]:
submission_df["status_group"] = test_preds

In [25]:
submission_df.head()

,status_group
id,
50785,non functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [26]:
submission_df.to_csv('submissions/V5.csv', index=True)